# OPTaaS Tutorial

OPTaaS is an optimization service developed by Mind Foundry. It can be used to optimize any function you wish to use; all it needs to know is the parameter space over which the function operates.

The procedure is simple: you create a task, which is the definition of the parameters of the function, and send it to OPTaaS. The service will then suggest a configuration (set of values for your parameters) to evaluate. You evaluate the configuration locally and send the score you obtained back to OPTaaS. You then repeat the process until you're happy that the score is good, or that there hasn't been enough progress in a while. 

Here we demonstrate how to use OPTaaS to optimize a scikit-learn classification pipeline.

## Load your dataset

We will run a classification pipeline using the German Credit Data available [here](https://newonlinecourses.science.psu.edu/stat857/node/215/). The data contains 1000 rows, with 20 feature columns and 1 target column which includes 2 classes.

In [1]:
import pandas as pd

data = pd.read_csv('german_credit.csv')
features = data[data.columns.drop(['Creditability'])]
target = data['Creditability']

## Create your pipeline

We can then assemble a simple pipeline: we reduce the input dimensions with PCA, and then classify the data with a Random Forest Classifier.

We create a function that will set the parameters of our pipeline based on the values that OPTaaS will generate.

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

def make_pipeline(**params):
    pipeline = Pipeline([('pca', PCA()), ('rf', RandomForestClassifier())])
    pipeline.set_params(**params)
    return pipeline

make_pipeline()

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
          ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

## Define the parameters to be optimized

Let's now define the parameters for our task. A list of available parameter types can be found [here](autogen/mindfoundry.optaas.client.html#module-mindfoundry.optaas.client.parameter).

We can use *constraints* to make sure that the parameter values are always valid, e.g. that `max_features` in our classifier will never be larger than `n_components` in PCA. (see also: [Constraints](constraints.ipynb))

scikit-learn pipelines follow a naming convention, so we abide to that convention here. This allows us to plug them in directly into `Pipeline.set_params()`.

Note that you don't have to define every single parameter of your pipelines, just the ones you would like to optimize. If you don't know what parameters will lead to better results, give them all to OPTaaS and let it do the hard work for you. However, the more parameters you have, the harder it is to optimize over them, so you will need more iterations to achieve comparable results. 

In [3]:
from mindfoundry.optaas.client.parameter import IntParameter, BoolParameter, CategoricalParameter, Distribution
from mindfoundry.optaas.client.constraint import Constraint

feature_count = len(features.columns)
n_components = IntParameter('pca__n_components', minimum=1, maximum=feature_count)
max_features = IntParameter('rf__max_features', minimum=1, maximum=feature_count, optional=True, include_in_default=False)
constraint = Constraint(when=max_features.is_present(), then=max_features < n_components) 

parameters = [
    # PCA parameters
    n_components,
    BoolParameter('pca__whiten'),
    
    # Random Forest parameters
    max_features,
    IntParameter('rf__n_estimators', minimum=10, maximum=1000, default=10, distribution=Distribution.LOGUNIFORM),
    CategoricalParameter('rf__criterion', values=['entropy', 'gini']),
    IntParameter('rf__max_depth', minimum=1, maximum=100, optional=True),
    IntParameter('rf__min_samples_split', minimum=2, maximum=20),
    IntParameter('rf__min_samples_leaf', minimum=1, maximum=20),
]

display(parameters)

[{'id': '2818768346080', 'name': 'pca__n_components', 'type': 'integer', 'minimum': 1, 'maximum': 20},
 {'id': '2818768505824', 'name': 'pca__whiten', 'type': 'boolean'},
 {'id': '2818768346472', 'name': 'rf__max_features', 'type': 'integer', 'optional': True, 'includeInDefault': False, 'minimum': 1, 'maximum': 20},
 {'id': '2818768505880', 'name': 'rf__n_estimators', 'type': 'integer', 'default': 10, 'minimum': 10, 'maximum': 1000, 'distribution': 'LogUniform'},
 {'id': '2818768505936', 'name': 'rf__criterion', 'type': 'categorical', 'enum': ['entropy', 'gini']},
 {'id': '2818768505992', 'name': 'rf__max_depth', 'type': 'integer', 'optional': True, 'minimum': 1, 'maximum': 100},
 {'id': '2818768506048', 'name': 'rf__min_samples_split', 'type': 'integer', 'minimum': 2, 'maximum': 20},
 {'id': '2818768506104', 'name': 'rf__min_samples_leaf', 'type': 'integer', 'minimum': 1, 'maximum': 20}]

## Connect to the OPTaaS server using your API Key

We now create a client, and connect to the web service that will perform our optimization. You will need to input your personal API key. Make sure you keep your key private and don't commit it to your version control system. 

In [4]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

## Create your Task

In [5]:
task = client.create_task(
    title='My Pipeline Optimization', 
    parameters=parameters,
    constraints=[constraint]
)

## Generate the first batch of configurations

Here OPTaaS gives us the first 5 configurations (sets of parameter values) for our problem.

In [6]:
configurations = task.generate_configurations(5)

display(configurations)

[{ 'id': 'b28e2056-b957-4838-85d5-93cd9856053f',
   'type': 'default',
   'values': { 'pca__n_components': 10,
               'pca__whiten': False,
               'rf__criterion': 'entropy',
               'rf__max_depth': 50,
               'rf__min_samples_leaf': 10,
               'rf__min_samples_split': 11,
               'rf__n_estimators': 10}},
 { 'id': '12d16ead-41fc-407f-897b-35ad2dc2a6c7',
   'type': 'exploration',
   'values': { 'pca__n_components': 4,
               'pca__whiten': True,
               'rf__criterion': 'gini',
               'rf__min_samples_leaf': 2,
               'rf__min_samples_split': 13,
               'rf__n_estimators': 46}},
 { 'id': '8b3e7c4f-5300-41a5-95f1-e2b0a7b4bdc6',
   'type': 'exploration',
   'values': { 'pca__n_components': 16,
               'pca__whiten': True,
               'rf__criterion': 'entropy',
               'rf__max_depth': 57,
               'rf__min_samples_leaf': 10,
               'rf__min_samples_split': 4,
            

## Use the configurations to set pipeline parameters and calculate the result

We can plug this configuration into our pipeline, run it and score it. We wrapped this into a function so that we can call it multiple times.

In [7]:
from sklearn.model_selection import cross_val_score

def get_result(configuration):
    pipeline = make_pipeline(**configuration.values)
    scores = cross_val_score(pipeline, features, target, scoring='f1_micro')
    mean_score = scores.mean()
    return list(scores), mean_score

scores, mean_score = get_result(configurations[0])

display(f'Mean score: {mean_score:.3f}')

'Mean score: 0.689'

## Record the result in OPTaaS and generate the next configuration to try

The score we obtained must now be passed back to OPTaaS, which will use it to generate and return a new configuration.

You can store additional user-defined data along with your result, but that's entirely optional and for your convenience only. For example, you can store the individual scores for each category so you can easily reference them later.

In [8]:
next_configuration = task.record_result(configurations[0], score=mean_score, user_defined_data=scores)
configurations.append(next_configuration)

## Repeat as necessary

Repeat the process until you're happy with the result or you've run out of time. How many times should we run it? There is no single answer to this question, but you can look for a certain amount of improvement at each iteration, and stop when you haven't improved in a while. Again this is up to you, and depending on how long you are prepared to wait and perhaps even on how much it costs to run each evaluation on your hardware, especially if you're using a cloud computing service. The more the better (remembering that harder problems, i.e. problems with more parameters, will need more iterations to find a decent solution).

In [9]:
number_of_iterations = 20

for i in range(1, number_of_iterations):
    configuration = configurations[i]
    display("OPTaaS recommendation:", configuration)
    
    scores, mean_score = get_result(configuration)
    display(f'Mean score: {mean_score:.3f}')
    
    next_configuration = task.record_result(configuration, score=mean_score, user_defined_data=scores)
    configurations.append(next_configuration)

'OPTaaS recommendation:'

{ 'id': '12d16ead-41fc-407f-897b-35ad2dc2a6c7',
  'type': 'exploration',
  'values': { 'pca__n_components': 4,
              'pca__whiten': True,
              'rf__criterion': 'gini',
              'rf__min_samples_leaf': 2,
              'rf__min_samples_split': 13,
              'rf__n_estimators': 46}}

'Mean score: 0.720'

'OPTaaS recommendation:'

{ 'id': '8b3e7c4f-5300-41a5-95f1-e2b0a7b4bdc6',
  'type': 'exploration',
  'values': { 'pca__n_components': 16,
              'pca__whiten': True,
              'rf__criterion': 'entropy',
              'rf__max_depth': 57,
              'rf__min_samples_leaf': 10,
              'rf__min_samples_split': 4,
              'rf__n_estimators': 64}}

...

{ 'id': '49adb2dc-e7ea-45c6-9c84-ed5871d280cb',
  'type': 'exploitation',
  'values': { 'pca__n_components': 4,
              'pca__whiten': True,
              'rf__criterion': 'gini',
              'rf__max_features': 1,
              'rf__min_samples_leaf': 2,
              'rf__min_samples_split': 12,
              'rf__n_estimators': 54}}

'Mean score: 0.698'

'OPTaaS recommendation:'

{ 'id': '8dd67531-1e0a-462a-be1e-c15e3276cd92',
  'type': 'exploitation',
  'values': { 'pca__n_components': 4,
              'pca__whiten': True,
              'rf__criterion': 'gini',
              'rf__min_samples_leaf': 1,
              'rf__min_samples_split': 4,
              'rf__n_estimators': 63}}

'Mean score: 0.708'

## Complete your task

Tell OPTaaS that you're satisfied and you won't run any more experiments, then go ahead and get your final result out. Job done!

In [10]:
task.complete()

best_result, best_configuration = task.get_best_result_and_configuration()

display(best_result, best_configuration)

{ 'configuration': '12d16ead-41fc-407f-897b-35ad2dc2a6c7',
  'id': 946,
  'score': 0.7199864535193875,
  'user_defined_data': [ 0.7335329341317365,
                         0.7147147147147147,
                         0.7117117117117117]}

{ 'id': '12d16ead-41fc-407f-897b-35ad2dc2a6c7',
  'type': 'exploration',
  'values': { 'pca__n_components': 4,
              'pca__whiten': True,
              'rf__criterion': 'gini',
              'rf__min_samples_leaf': 2,
              'rf__min_samples_split': 13,
              'rf__n_estimators': 46}}